In [1]:
import os
import pandas as pd
from pandas_profiling import ProfileReport

In [2]:
try:
    _ = first_run
except NameError:
    first_run = True
    os.chdir(os.getcwd().rsplit("/", 1)[0])

# Load Data

In [9]:
df = pd.read_csv("../data/train/X_train.csv", index_col=0).join(pd.read_csv("../data/train/y_train.csv", index_col=0))

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71980 entries, 27330 to 16866
Data columns (total 43 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   uuid                                 71980 non-null  object 
 1   account_amount_added_12_24m          71980 non-null  int64  
 2   account_days_in_dc_12_24m            63440 non-null  float64
 3   account_days_in_rem_12_24m           63440 non-null  float64
 4   account_days_in_term_12_24m          63440 non-null  float64
 5   account_incoming_debt_vs_paid_0_24m  29351 non-null  float64
 6   account_status                       32880 non-null  float64
 7   account_worst_status_0_3m            32880 non-null  float64
 8   account_worst_status_12_24m          23962 non-null  float64
 9   account_worst_status_3_6m            30480 non-null  float64
 10  account_worst_status_6_12m           28565 non-null  float64
 11  age                     

In [11]:
profile = ProfileReport(
    df,
    title="Basic Profile Report - Klarna Dataset",
    explorative=True,
    pool_size=8,
    sort="ascending",
    vars={
        "cat": {
            "words": True,
            "cardinality_threshold": 10
        }
    },
    html={
        "style":{
            "full_width": True
        }
    }

)

In [12]:
profile.to_file("1_sanity/basic_profile.html")

Summarize dataset:  82%|████████▏ | 47/57 [00:15<00:05,  1.95it/s, Calculate kendall correlation] /home/ottok92/miniconda3/envs/klarna/lib/python3.8/site-packages/scipy/stats/stats.py:4594: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
Export report to file: 100%|██████████| 1/1 [00:00<00:00, 11.99it/s]
